In [142]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import random
from statistics import mode
import re
from nltk.tokenize import RegexpTokenizer
from collections import Counter
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from sklearn.feature_extraction.text import CountVectorizer
from gensim.corpora import Dictionary
%config InlineBackend.figure_format = 'retina'

In [6]:
df = pd.read_csv('data/chrisbrown.csv')
df

,api_path,primary_artist,title,url,song_endpoint,album,release_date,lyrics
0,/songs/62387,Chris Brown,100%,https://genius.com/Chris-brown-100-lyrics,http://api.genius.com/songs/62387,Boy In Detention (Mixtape),2011-08-05,"Beats I be servin', Saks Fifth splurgin' Diam..."
1,/songs/54007,Chris Brown,100 Bottles,https://genius.com/Chris-brown-100-bottles-lyrics,http://api.genius.com/songs/54007,Boy In Detention (Mixtape),2011-08-05,We in the motherfuckin' buildin' 100 fuckin...
2,/songs/516349,Chris Brown,101 (Interlude),https://genius.com/Chris-brown-101-interlude-l...,http://api.genius.com/songs/516349,X,2014-09-16,You got me doing 101 on the 101 Don't go nowh...
3,/songs/3103904,Chris Brown,10 Feet,https://genius.com/Chris-brown-10-feet-lyrics,http://api.genius.com/songs/3103904,None,2017-05-31,Ain't no way we could ignore this fire When I...
4,/songs/57217,Chris Brown,2011 BET Cypher Chris Brown and Friends,https://genius.com/Chris-brown-2011-bet-cypher...,http://api.genius.com/songs/57217,None,2019-04-04,"Welcome to the Cypher Myself, K-Mac, Ace Hood..."
5,/songs/79425,Chris Brown,2012,https://genius.com/Chris-brown-2012-lyrics,http://api.genius.com/songs/79425,Fortune,2012-06-29,Faire l'amour lendemain What if the sky was...
6,/songs/60609,Chris Brown,2 Complicated,https://genius.com/Chris-brown-2-complicated-l...,http://api.genius.com/songs/60609,None,2012-06-19,Shattered glasses and cigarettes There's a lo...
7,/songs/3089879,Chris Brown,2 the Side,https://genius.com/Chris-brown-2-the-side-lyrics,http://api.genius.com/songs/3089879,None,2017-05-28,"Oh, oh, oh, babe, yeah I know what you need, ..."
8,/songs/2404874,Chris Brown,45,https://genius.com/Chris-brown-45-lyrics,http://api.genius.com/songs/2404874,None,2017-05-31,And I'm like Jordan wearin' 45 I told you I w...
9,/songs/2877,Chris Brown,48 Bar Rap,https://genius.com/Chris-brown-48-bar-rap-lyrics,http://api.genius.com/songs/2877,Fan of a Fan,2010-05-16,"Yeah, leggo It's what they wanted right? Bl..."


In [143]:
stopwords = stopwords.words('english')
stopwords.extend(['don', 'like', 'ain', 'oh', 'll', 'ooh', 'na', 'just', 've', 'tha', 'yeah', 'gon'])

In [144]:
def prepare_corpus(df):
    ''' Prepare the corpus from the artists lyrics by tokenizing and lemmatizing the words'''
    corpus = []
    tokenizer = RegexpTokenizer(r'\w\w+')
    lemmatizer = WordNetLemmatizer()
    for lyric in df['lyrics']:
        token = tokenizer.tokenize(str(lyric).lower())
        lemmatized_token = []
        for word in token:
            if word not in stopwords:
                lemmatized_word = lemmatizer.lemmatize(word)
                lemmatized_token.append(lemmatized_word)
        corpus.append(' '.join(lemmatized_token))
    print('Prepared the corpus')
    return corpus

In [145]:
corpus = prepare_corpus(df)

Prepared the corpus


In [146]:
corpus = pd.DataFrame(corpus, columns=['corpus'])

In [147]:
vectorizer = CountVectorizer()
dtm = vectorizer.fit_transform(corpus['corpus'])
print(dtm.shape)

(588, 7084)


In [151]:
df_v = pd.DataFrame(dtm.todense(), columns=vectorizer.get_feature_names())

In [152]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
model = LDA(n_components=3, random_state=0)
model.fit(dtm)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=3, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [153]:
model.components_

array([[  2.33100111,  16.17848865, 135.11270035, ...,   0.3333362 ,
          0.33333604,   0.3333362 ],
       [  0.33333995,   5.30176415,   1.54679086, ...,   0.33333562,
          0.33333549,   0.33333562],
       [  0.33565894,   8.5197472 ,   0.34050879, ...,   1.33332819,
          2.33332847,   1.33332819]])

In [157]:
pd.DataFrame(model.components_[0], columns=vectorizer.get_feature_names()).T

ValueError: Shape of passed values is (7084, 1), indices imply (7084, 7084)

In [154]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()